In [ ]:
import os
import cv2
import glob
import ntpath
import numpy
import pandas

In [ ]:
make_negative_cube()

In [ ]:
def make_negative_cube():

    dst_path = "G:/LungCancerPredict/generated/luna16_negative_cube/"

    os.makedirs(dst_path, exist_ok=True)

    # candidates = 'luna16官方提供假阳性'、fp = '简单算法检测假阳性‘、tn = '来自肺边界部位的随机采样‘
    src_paths = ['G:/LungCancerPredict/resources/luna16_falsepos_labels/*_candidates_falsepos.csv',
                 'G:/LungCancerPredict/extracted/luna16_extracted_images/_labels/*_fp_nodule.csv',
                 'G:/LungCancerPredict/extracted/luna16_extracted_images/_labels/*_tn_nodule.csv']

    for src_path in src_paths:

        for index, csv_file in enumerate(glob.glob(src_path)):

            patient_id = ntpath.basename(csv_file).split('_')[0]
            negative_type = ntpath.basename(csv_file).split('_')[1]
            
            df_labels = pandas.read_csv(csv_file)
          
            #如果没有label信息，则跳过
            if len(df_labels) == 0: continue
            
            #读取该病人重采样后的图片
            imgs = load_patient_images(patient_id)
        
            #遍历该病人的label
            row_no = 0
            for index, row in df_labels.iterrows():
    
                coord_x = int(row["coord_x"] * imgs.shape[2])
                coord_y = int(row["coord_y"] * imgs.shape[1])
                coord_z = int(row["coord_z"] * imgs.shape[0])
                counter = int(row["anno_index"])
                
                #以坐标为中心创建一个48*48*48的cube
                cube_img = get_cube_from_img(imgs, coord_x, coord_y, coord_z, 48)
                if cube_img.sum() < 10: continue
    
                #将该cube数据以6*8个，48*48的图片形式保存
                save_cube_img(dst_path + patient_id + "_" + str(counter) + "_0_" + negative_type +".png", cube_img, 6, 8)
               
                #控制生成cube的数量上限
                row_no += 1
                max_item = 200
                if candidate_type == "fp": max_item = 500
                if row_no > max_item: break

def load_patient_images(patient_id):

    src_path = "G:/LungCancerPredict/extracted/luna16_extracted_images/" + patient_id + "/"
    src_files = glob.glob(src_path + "*_i.png")
    
    src_files.sort()
    
    imgs = [cv2.imread(src_file, cv2.IMREAD_GRAYSCALE) for src_file in src_files]
    imgs = [img.reshape((1, ) + img.shape) for img in imgs]
    
    res = numpy.vstack(imgs)
    
    return res

def get_cube_from_img(img3d, center_x, center_y, center_z, block_size):

    start_x = max(center_x - block_size / 2, 0) if center_x + block_size / 2 <= img3d.shape[2] else img3d.shape[2] - block_size
    start_y = max(center_y - block_size / 2, 0) if center_y + block_size / 2 <= img3d.shape[1] else img3d.shape[1] - block_size
    start_z = max(center_z - block_size / 2, 0) if center_z + block_size / 2 <= img3d.shape[0] else img3d.shape[0] - block_size

    start_z = int(start_z)
    start_y = int(start_y)
    start_x = int(start_x)
    
    res = img3d[start_z:start_z + block_size, start_y:start_y + block_size, start_x:start_x + block_size]
    
    return res

def save_cube_img(dst_path, cube_img, rows, cols):
    
    img_height = cube_img.shape[1]
    img_width = cube_img.shape[1]
    res_img = numpy.zeros((rows * img_height, cols * img_width), dtype=numpy.uint8)

    for row in range(rows):
        
        for col in range(cols):
            
            target_y = row * img_height
            target_x = col * img_width
            
            res_img[target_y:target_y + img_height, target_x:target_x + img_width] = cube_img[row * cols + col]

    cv2.imwrite(dst_path, res_img)
